In [1]:
!pip install mapie -q
!pip install catboost optuna -q

In [14]:
# data handling
import numpy as np
import pandas as pd

# Get multiple outputs from one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Train/Test splitting
from sklearn.model_selection import train_test_split

# tuning
import optuna

# Modeling
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from mapie.regression import MapieRegressor


# Ignore warning messages
import warnings
warnings.filterwarnings('ignore')

# Birth Weight Prediction

## Strategy

## Data exploration
- Upload the data ✅
- Descriptive statistics ✅
- Resort columns 
- Cross-features correlations

In [3]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [4]:
# Describing the dataset

print('--------------')
print('Rows, Columns:')
print('--------------')
df.shape
print('---------------')
print("Columns' names:")
print('---------------')
df.columns
print('-----------------')
print('Column / Datatype:')
print('-----------------')
df.dtypes
print('-----------------')
df.head(5)
print('---------------------')
print('Descriptive statistic:')
print('---------------------')
round(df.describe())
print('-----------------------')
print('The sum of null values:')
print('-----------------------')
print(df.isnull().sum())
print('-----------------------')
print('The sum of NaN values:')
print('-----------------------')
df.isna().sum()

--------------
Rows, Columns:
--------------


(108082, 38)

---------------
Columns' names:
---------------


Index(['id', 'ATTEND', 'BFACIL', 'BMI', 'CIG_0', 'DLMP_MM', 'DMAR', 'DOB_MM',
       'DOB_TT', 'DOB_WK', 'FAGECOMB', 'FEDUC', 'ILLB_R', 'ILOP_R', 'ILP_R',
       'LD_INDL', 'MAGER', 'MBSTATE_REC', 'MEDUC', 'M_Ht_In', 'NO_INFEC',
       'NO_MMORB', 'NO_RISKS', 'PAY', 'PAY_REC', 'PRECARE', 'PREVIS',
       'PRIORDEAD', 'PRIORLIVE', 'PRIORTERM', 'PWgt_R', 'RDMETH_REC',
       'RESTATUS', 'RF_CESAR', 'RF_CESARN', 'SEX', 'WTGAIN', 'DBWT'],
      dtype='object')

-----------------
Column / Datatype:
-----------------


...

,id,ATTEND,BFACIL,BMI,CIG_0,DLMP_MM,DMAR,DOB_MM,DOB_TT,DOB_WK,...,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN,DBWT
0,0,1,1,18.5,0,12,,10,1434,5,...,0,0,108,1,1,N,0,F,24,2800
1,1,1,1,18.3,2,4,1,12,2156,6,...,2,1,100,1,1,N,0,M,18,1900
2,2,1,1,27.3,0,3,2,12,1241,2,...,2,2,135,4,1,Y,2,F,27,2960
3,3,1,1,24.0,0,7,2,4,1649,2,...,0,0,111,3,1,N,0,M,29,3657
4,4,2,1,23.6,0,6,1,3,752,2,...,2,0,121,4,1,Y,2,F,37,3742


---------------------
Descriptive statistic:
---------------------


,id,ATTEND,BFACIL,BMI,CIG_0,DLMP_MM,DOB_MM,DOB_TT,DOB_WK,FAGECOMB,...,PREVIS,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESARN,WTGAIN,DBWT
count,108082.0,108082.0,108082.0,108082.0,108082.0,108082.0,108082.0,108082.0,108082.0,108082.0,...,108082.0,108082.0,108082.0,108082.0,108082.0,108082.0,108082.0,108082.0,108082.0,108082.0
mean,54040.0,1.0,1.0,29.0,2.0,11.0,7.0,1233.0,4.0,40.0,...,14.0,0.0,1.0,1.0,176.0,2.0,1.0,0.0,32.0,3260.0
std,31201.0,1.0,0.0,13.0,8.0,20.0,3.0,633.0,2.0,22.0,...,14.0,5.0,4.0,5.0,125.0,1.0,1.0,2.0,19.0,590.0
min,0.0,1.0,1.0,13.0,0.0,1.0,1.0,0.0,1.0,14.0,...,0.0,0.0,0.0,0.0,75.0,1.0,1.0,0.0,0.0,227.0
25%,27020.0,1.0,1.0,22.0,0.0,4.0,4.0,801.0,2.0,28.0,...,9.0,0.0,0.0,0.0,130.0,1.0,1.0,0.0,20.0,2965.0
50%,54040.0,1.0,1.0,26.0,0.0,7.0,7.0,1238.0,4.0,33.0,...,12.0,0.0,1.0,0.0,150.0,1.0,1.0,0.0,30.0,3300.0
75%,81061.0,1.0,1.0,31.0,0.0,10.0,10.0,1735.0,6.0,38.0,...,14.0,0.0,2.0,1.0,182.0,3.0,2.0,0.0,40.0,3629.0
max,108081.0,9.0,9.0,100.0,99.0,99.0,12.0,9999.0,7.0,99.0,...,99.0,99.0,99.0,99.0,999.0,9.0,4.0,99.0,99.0,6840.0


-----------------------
The sum of null values:
-----------------------
id             0
ATTEND         0
BFACIL         0
BMI            0
CIG_0          0
DLMP_MM        0
DMAR           0
DOB_MM         0
DOB_TT         0
DOB_WK         0
FAGECOMB       0
FEDUC          0
ILLB_R         0
ILOP_R         0
ILP_R          0
LD_INDL        0
MAGER          0
MBSTATE_REC    0
MEDUC          0
M_Ht_In        0
NO_INFEC       0
NO_MMORB       0
NO_RISKS       0
PAY            0
PAY_REC        0
PRECARE        0
PREVIS         0
PRIORDEAD      0
PRIORLIVE      0
PRIORTERM      0
PWgt_R         0
RDMETH_REC     0
RESTATUS       0
RF_CESAR       0
RF_CESARN      0
SEX            0
WTGAIN         0
DBWT           0
dtype: int64
-----------------------
The sum of NaN values:
-----------------------


id             0
ATTEND         0
BFACIL         0
BMI            0
CIG_0          0
DLMP_MM        0
DMAR           0
DOB_MM         0
DOB_TT         0
DOB_WK         0
FAGECOMB       0
FEDUC          0
ILLB_R         0
ILOP_R         0
ILP_R          0
LD_INDL        0
MAGER          0
MBSTATE_REC    0
MEDUC          0
M_Ht_In        0
NO_INFEC       0
NO_MMORB       0
NO_RISKS       0
PAY            0
PAY_REC        0
PRECARE        0
PREVIS         0
PRIORDEAD      0
PRIORLIVE      0
PRIORTERM      0
PWgt_R         0
RDMETH_REC     0
RESTATUS       0
RF_CESAR       0
RF_CESARN      0
SEX            0
WTGAIN         0
DBWT           0
dtype: int64

## Data Preparation
- Reduce skewness

## Feature Engineering
- Creating new values
- One-hot encoding
- Dropping the unnecessary
- Scaling

## Splitting the dataset
- splitting the train set to validation

In [5]:
# splitting df dataset into train and validation
# Dropping the categorical data to try the model

X = df.drop(['DBWT', 'DMAR', 'LD_INDL', 'RF_CESAR', 'SEX'], axis=1)
y = df['DBWT']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

## Evaluating Models
- Winkler Score

In [6]:
# Winkler scoring

# Notice that we use absolute values due to the possibility of 'quantile crossing' where lower > upper.
def WIS_and_coverage(y_true,lower,upper,alpha):

    assert np.isnan(y_true) == False, "y_true contains NaN value(s)"
    assert np.isinf(y_true) == False, "y_true contains inf values(s)"
    assert np.isnan(lower)  == False, "lower interval value contains NaN value(s)"
    assert np.isinf(lower)  == False, "lower interval value contains inf values(s)"
    assert np.isnan(upper)  == False, "upper interval value contains NaN value(s)"
    assert np.isinf(upper)  == False, "upper interval value contains inf values(s)"
    assert alpha > 0 and alpha <= 1,  f"alpha should be (0,1]. Found: {alpha}"

    # WIS for one single row
    score = np.abs(upper-lower)
    if y_true < np.minimum(upper,lower):
        score += ((2/alpha) * (np.minimum(upper,lower) - y_true))
    if y_true > np.maximum(upper,lower):
        score += ((2/alpha) * (y_true - np.maximum(upper,lower)))
    # coverage for one single row
    coverage  = 1 # assume is within coverage
    if (y_true < np.minimum(upper,lower)) or (y_true > np.maximum(upper,lower)):
        coverage = 0
    return score, coverage

# vectorize the function
v_WIS_and_coverage = np.vectorize(WIS_and_coverage)

def score(y_true,lower,upper,alpha):
    """
    This is an implementation of the Winkler Interval score (https://otexts.com/fpp3/distaccuracy.html#winkler-score).
    The mean over all of the individual Winkler Interval scores (MWIS) is returned, along with the coverage.

    See:
    [1] Robert L. Winkler "A Decision-Theoretic Approach to Interval Estimation", Journal of the American Statistical Association, vol. 67, pp. 187-191 (1972) (https://doi.org/10.1080/01621459.1972.10481224)
    [2] Tilmann Gneiting and Adrian E Raftery "Strictly Proper Scoring Rules, Prediction, and Estimation", Journal of the American Statistical Association, vol. 102, pp. 359-378 (2007) (https://doi.org/10.1198/016214506000001437) (Section 6.2)

    Version: 1.0.4
    Author:  Carl McBride Ellis
    Date:    2023-12-07
    """

    assert y_true.ndim == 1, "y_true: pandas Series or 1D array expected"
    assert lower.ndim  == 1, "lower: pandas Series or 1D array expected"
    assert upper.ndim  == 1, "upper: pandas Series or 1D array expected"
    assert isinstance(alpha, float) == True, "alpha: float expected"

    WIS_scores, coverage = v_WIS_and_coverage(y_true,lower,upper,alpha)
    MWIS      = np.mean(WIS_scores)
    MWIS      = float(MWIS)
    coverage  = coverage.sum()/coverage.shape[0]
    coverage  = float(coverage)

    return MWIS,coverage

alpha = 0.1

## Modeling
- Hyperparameter tuning
- Random Forest
- XGBoost
- CatBoost

### Random-forest

In [19]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

predictions = rf_model.predict(X_test)
prediction_intervals = np.quantile(predictions, [alpha / 2, 1 - alpha / 2], axis=0)

results_df = pd.DataFrame({
    'y_true': y_test.values,
    'lower': prediction_intervals[0],
    'upper': prediction_intervals[1]
})

# MWIS score
MWIS, coverage = score(results_df["y_true"], results_df["lower"], results_df["upper"], alpha)

# Print MWIS score and coverage
print("MWIS score:", round(MWIS, 3))
print("Coverage:", round(coverage * 100, 1), "%")

RandomForestRegressor(random_state=42)

MWIS score: 4054.473
Coverage: 55.9 %


### XGBoost x Optuna

In [15]:
# def objective(trial):
#     params = {
#         "n_estimators": 1000,
#         "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
#         "max_depth": trial.suggest_int("max_depth", 1, 10),
#         "subsample": trial.suggest_float("subsample", 0.05, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
#         "min_child_weight": trial.suggest_int("min_child_weight", 1, 100),
#     }

#     model = XGBRegressor(**params, silent=True)
#     model.fit(X_train, y_train)
#     predictions = model.predict(X_test)
#     prediction_intervals = np.quantile(predictions, [alpha / 2, 1 - alpha / 2], axis=0)
#     results_df = pd.DataFrame({
#         'y_true': y_test.values,
#         'lower': prediction_intervals[0],
#         'upper': prediction_intervals[1]
#     })
#     MWIS, coverage = score(results_df["y_true"], results_df["lower"], results_df["upper"], alpha)
#     return MWIS

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

# # Retrieve the best parameters and best value from the study
# best_params_xgboost = study.best_params
# best_score_xgboost = study.best_value

# print("Best XGBoost parameters:", best_params_xgboost)
# print("Best XGBoost MWIS score:", round(best_score_xgboost, 3))

[I 2024-02-26 18:52:05,187] A new study created in memory with name: no-name-0ea9ae50-51ae-482f-aa3e-3dfd766ab988
[I 2024-02-26 18:52:09,813] Trial 0 finished with value: 4718.199490359691 and parameters: {'learning_rate': 0.01684048779008674, 'max_depth': 2, 'subsample': 0.657851577613668, 'colsample_bytree': 0.14886811516954576, 'min_child_weight': 93}. Best is trial 0 with value: 4718.199490359691.
[I 2024-02-26 18:52:15,917] Trial 1 finished with value: 4358.019560257299 and parameters: {'learning_rate': 0.008187111814620887, 'max_depth': 4, 'subsample': 0.8481300651129258, 'colsample_bytree': 0.7657722547133762, 'min_child_weight': 99}. Best is trial 1 with value: 4358.019560257299.
[I 2024-02-26 18:52:21,982] Trial 2 finished with value: 4200.027789053375 and parameters: {'learning_rate': 0.01268818970883468, 'max_depth': 4, 'subsample': 0.9736776398458722, 'colsample_bytree': 0.337550925424914, 'min_child_weight': 40}. Best is trial 2 with value: 4200.027789053375.
[I 2024-02-26

Best XGBoost parameters: {'learning_rate': 0.09185805664059714, 'max_depth': 10, 'subsample': 0.10627511169638384, 'colsample_bytree': 0.8679770086150229, 'min_child_weight': 5}
Best XGBoost MWIS score: 2758.544


NameError: name 'xgb' is not defined

In [18]:
# Train the XGBoost model with the best parameters found
model = XGBRegressor(learning_rate=0.09185805664059714, 
                     max_depth=10, 
                     subsample=0.10627511169638384, 
                     colsample_bytree=0.8679770086150229, 
                     min_child_weight=5)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
prediction_intervals = np.quantile(predictions, [alpha / 2, 1 - alpha / 2], axis=0)
results_df = pd.DataFrame({
    'y_true': y_test.values,
    'lower': prediction_intervals[0],
    'upper': prediction_intervals[1]
})
MWIS, coverage = score(results_df["y_true"], results_df["lower"], results_df["upper"], alpha)
print("MWIS score:", round(MWIS, 3))
print("Coverage:", round(coverage * 100, 1), "%")

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8679770086150229, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.09185805664059714, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

MWIS score: 3527.016
Coverage: 64.8 %


### CatBoost x Optuna

In [7]:
# def objective(trial):
#     params = {
#         "iterations": 1000,
#         "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
#         "depth": trial.suggest_int("depth", 1, 20),
#         "subsample": trial.suggest_float("subsample", 0.05, 1.0),
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
#     }

#     model = CatBoostRegressor(**params, silent=True)
#     model.fit(X_train, y_train)
#     predictions = model.predict(X_test)
#     prediction_intervals = np.quantile(predictions, [alpha / 2, 1 - alpha / 2], axis=0)
#     results_df = pd.DataFrame({
#         'y_true': y_test.values,
#         'lower': prediction_intervals[0],
#         'upper': prediction_intervals[1]
#     })
#     MWIS, coverage = score(results_df["y_true"], results_df["lower"], results_df["upper"], alpha)
#     return MWIS

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

# best_params_catboost = study.best_params
# best_score_catboost = study.best_value

# print("Best CatBoost parameters:", best_params_catboost)
# print("Best CatBoost MWIS score:", round(best_score_catboost, 3))

# # Calculate MWIS score for the best parameters found
# model = CatBoostRegressor(**best_params_catboost, silent=True)
# model.fit(X_train, y_train)
# predictions = model.predict(X_test)
# prediction_intervals = np.quantile(predictions, [alpha / 2, 1 - alpha / 2], axis=0)
# results_df = pd.DataFrame({
#     'y_true': y_test.values,
#     'lower': prediction_intervals[0],
#     'upper': prediction_intervals[1]
# })
# MWIS, coverage = score(results_df["y_true"], results_df["lower"], results_df["upper"], alpha)
# print("MWIS score:", round(MWIS, 3))
# print("Coverage:", round(coverage * 100, 1), "%")


[I 2024-02-26 17:49:09,311] A new study created in memory with name: no-name-57e30bf1-e40f-45f5-8b84-03897df87208
[I 2024-02-26 17:49:30,052] Trial 0 finished with value: 4596.503531686753 and parameters: {'learning_rate': 0.004761640779102068, 'depth': 8, 'subsample': 0.38100433240553305, 'colsample_bylevel': 0.7802960890718474, 'min_data_in_leaf': 39}. Best is trial 0 with value: 4596.503531686753.
[I 2024-02-26 17:49:37,267] Trial 1 finished with value: 3738.5075823514508 and parameters: {'learning_rate': 0.06957916159300247, 'depth': 3, 'subsample': 0.5073122204789776, 'colsample_bylevel': 0.9262351283511784, 'min_data_in_leaf': 63}. Best is trial 1 with value: 3738.5075823514508.
[I 2024-02-26 17:49:41,659] Trial 2 finished with value: 6885.065163114176 and parameters: {'learning_rate': 0.002948254263246553, 'depth': 1, 'subsample': 0.16491563824985017, 'colsample_bylevel': 0.36639558292577845, 'min_data_in_leaf': 71}. Best is trial 1 with value: 3738.5075823514508.
[I 2024-02-26 

Best CatBoost parameters: {'learning_rate': 0.09886907332732021, 'depth': 10, 'subsample': 0.9045876332454507, 'colsample_bylevel': 0.5525983482670428, 'min_data_in_leaf': 10}
Best CatBoost MWIS score: 3464.596


MWIS score: 3464.596
Coverage: 66.2 %


In [11]:
# CatBoost w/best parameters from optuna
catboost_model = CatBoostRegressor(learning_rate=0.09886907332732021, 
                                    depth=10, 
                                    subsample=0.9045876332454507, 
                                    colsample_bylevel=0.5525983482670428, 
                                    min_data_in_leaf=10)
catboost_model.fit(X_train, y_train, verbose=False)

predictions = catboost_model.predict(X_test)

# Compute prediction intervals
prediction_intervals = np.quantile(predictions, [alpha / 2, 1 - alpha / 2], axis=0)

results_df = pd.DataFrame({
    'y_true': y_test.values,
    'lower': prediction_intervals[0],
    'upper': prediction_intervals[1]
})

# MWIS score
MWIS, coverage = score(results_df["y_true"], results_df["lower"], results_df["upper"], alpha)
print("MWIS score:", round(MWIS, 3))
print("Coverage:", round(coverage * 100, 1), "%")

MWIS score: 3464.596
Coverage: 66.2 %


## Uploading results